## Get clusters with only one class

In [ ]:
train_f1 = r'G:\autocls_exp2\lcfilter\f1_lisboa.tif'

clusters = r'G:\autocls_exp2\cluster_filter\cmb_km_f1_lisboa.tif'

result = r'G:\autocls_exp2\lcfilter\f5_lisboa.tif'

In [ ]:
import os
import pandas as pd
import numpy as np
from glass.rst.local import combine
from glass.pys.oss import mkdir
from glass.pys.tm import now_as_str
from glass.rd import tbl_to_obj
from glass.esri.rst.local import combine_rst, lookup
from glass.wt import obj_to_tbl
from glass.esri.tbl.joins import join_table
from glass.esri.tbl.col import cols_calc
from glass.esri.rd.rst import rst_to_lyr
from glass.esri.rst.rcls import rcls_rst
from glass.esri.rd import dbf_to_view

In [ ]:
# Create Layers
tf1lyr = rst_to_lyr(train_f1, 'f1')
cltlyr = rst_to_lyr(clusters, 'clusters')

In [ ]:
cltlyr

In [ ]:
tmpws = mkdir(
    os.path.join(os.path.dirname(result), now_as_str(utc=True)),
    overwrite=True
)

In [ ]:
# Combine
clstbycls, clyr = combine_rst(
    [train_f1, clusters],
    os.path.join(tmpws, 'c_cls_cluster.tif'),
    train_f1
)

vatdbf = os.path.join(tmpws, f"{os.path.basename(clstbycls)}.vat.dbf")
df_clstbycls = tbl_to_obj(vatdbf)

In [ ]:
df_clstbycls

In [ ]:
class_col = df_clstbycls.columns.values[2]
cluster_col = df_clstbycls.columns.values[3]

# Count number of classes for each cluster
clstsum = pd.DataFrame({
    'nclasses' : df_clstbycls.groupby([cluster_col])[class_col].agg('count')
}).reset_index()

In [ ]:
clstsum['classv'] = np.where(
    clstsum.nclasses > 1,
    0, 1
)

In [ ]:
# Save table as DBF
clstnum_dbf = obj_to_tbl(clstsum, os.path.join(tmpws, 'ncls_by_cluster.dbf'))

In [ ]:
# Join Table
#_vatv = dbf_to_view(vatdbf, 'combinevatdbf')
_vatdbf = join_table(clyr, clstnum_dbf, cluster_col, cluster_col.upper(), cols=['CLASSV'])

# Calculate
_vatdbf = cols_calc(
    _vatdbf, "classid", f"getcls(!CLASSV!, !{class_col}!)",
    newcols={"TYPE": "SHORT", "LENGTH": None, "PRECISION" : None},
    code="""def getcls(mask, cls):
        if not mask:
            return 0
        return cls
    """
)

In [ ]:
# Lookup
#cmblyr = rst_to_lyr(clstbycls)
lookrst, looklyr = lookup(
    clyr, "classid",
    os.path.join(tmpws, 'lookuprst.tif'),
    train_f1
)

# Reclassify
lulc_classes = list(df_clstbycls[class_col].unique())
rcls_rules = [[cls, cls] for cls in lulc_classes]
rcls_rules.append([0, 'NODATA'])

rcls_rst(looklyr, 'Value', rcls_rules, result, train_f1)